In [1]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

import quant_utils.data_moudle as dm
from quant_utils.constant import DB_CONN_JJTG_DATA, DB_CONN_WIND
import data_functions.fund_data as dfs
import data_functions.portfolio_data as pds
from quant_utils.performance import Performance, periods_performance

In [4]:
# short_bond = pd.read_excel("D:/中债+美股.xlsx")
short_bond = dm.get_fund_index(ticker_symbol="885008")
short_bond = short_bond.set_index(['TRADE_DT'])

In [6]:
periods_performance(short_bond["S_DQ_CLOSE"]).to_excel("d:/tongji3.xlsx")

In [8]:
short_bond["S_DQ_CLOSE"]

TRADE_DT
20031231    1000.0000
20040102    1000.2531
20040105    1005.1117
20040106    1004.8210
20040107    1006.4898
              ...    
20240122    2401.7249
20240123    2401.5976
20240124    2401.9183
20240125    2402.7290
20240126    2403.2603
Name: S_DQ_CLOSE, Length: 4879, dtype: float64

In [7]:
Performance(short_bond["S_DQ_CLOSE"]).rolling_ret_stats()

D:\FundResearch\quant_utils\performance.py:592: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return pd.DataFrame.from_dict(perf_dict).applymap(lambda x: format(x, ".2%"))


,7天,1个月,2个月,3个月,6个月,9个月,1年,2年,3年,4年,5年
收益率25分位数,-0.03%,0.02%,0.17%,0.33%,1.14%,1.94%,2.74%,6.50%,10.76%,15.52%,20.71%
收益率中位数,0.12%,0.35%,0.68%,1.02%,2.04%,3.02%,4.18%,8.47%,13.00%,17.84%,24.80%
收益率75分位数,0.28%,0.68%,1.25%,1.81%,3.18%,4.40%,5.67%,11.03%,17.54%,23.72%,29.84%
胜率,71.78%,75.82%,80.80%,83.32%,88.42%,92.36%,96.40%,100.00%,100.00%,100.00%,100.00%


In [ ]:
dfs.get_fund_alpha_to_index("")

In [ ]:
df = dm.get_fund_index('885062').set_index("TRADE_DT")

In [ ]:
a = periods_performance(df["S_DQ_CLOSE"])

In [ ]:
a.to_excel("d:/885062分析.xlsx")

In [ ]:
df = dfs.query_fund_performance_rank_pct("20231228")
df = df.set_index(["TRADE_DT", "TICKER_SYMBOL", "LEVEL"]).stack().reset_index()
df.rename(columns={'level_3': "INDICATOR", 0:"VALUE"}, inplace=True)
df['TEMP'] = df["INDICATOR"].str.split("_")
df["CYCLE"] = df['TEMP'].apply(lambda x: x[-1])
df["INDICATOR"] = df['TEMP'].apply(lambda x: x[1])
df.drop(columns=["TEMP"], inplace=True)

In [ ]:
indicator_weight_dict = {
    'AVGRETURN': 0.4, 
    'STDARDDEV': 0.0, 
    'SHARPRATIO': 0.2, 
    'MAXDOWNSIDE': 0.2, 
    'CALMAR': 0.2,
}


In [ ]:
cycle_weight_dict = {
    'DAY': 0.0, 
    'THISYEAR':0.0, 
    'QUARTER': 0.0, 
    'HALFYEAR': 0.2, 
    'YEAR': 0.2, 
    'TWOYEAR': 0.3,
    'THREEYEAR': 0.3, 
    'FOURYEAR': 0.0, 
    'FIVEYEAR': 0.0, 
    'SIXYEAR': 0.0,
}

In [ ]:
df["INDICATOR"].unique()

In [ ]:
df["CYCLE"].unique()

In [ ]:
df = pd.read_excel("d:/现金及其等价物.xlsx", sheet_name="Sheet6")
df = df.set_index("TRADE_DT")

In [ ]:
df_ret = df[['黄金','短债','中证红利全收益', "偏股基金", "沪深300"]]

In [ ]:
df_ret.corr()

In [ ]:
df_nav = df[['黄金净值','短债净值','中证红利全收益净值', "偏股基金", "沪深300"]]

In [ ]:
perf_list = []
for i in df.columns:
    perf_list.append(Performance(df[i]).stats().T)
pd.concat(perf_list)

In [ ]:
temp = periods_performance(df["组合净值"])

In [ ]:
temp.query("PERIOD_NAME == 'ALL'")

In [ ]:
temp.query("PERIOD_NAME == '年度'")

In [ ]:
code_list_300 = [
    '163407',
    '008238',
    # '673100',
    '004190',
    '003015',
    "002837",
]
code_list_500 = [
'006730',
'007413',
'002872',
'009727',
'006104',
'009300',
# '007832',
# '502000'
]

code_list_1000 = [
'004194',
'015784',
'161039',
'006165',
'014125',
]

code_list_bond = [
'005725', '006797', '006902', '005754', '006965', '007245', '006629', '006989', '006597', '002301', '006319' 
]
result_list = []
for code in code_list_1000:
    df = dfs.get_fund_alpha_to_index(
        code, 
        index_code='000852',
        start_date='20220101'
    )
    result_list.append(df)
result = pd.concat(result_list)

In [ ]:
result.pivot_table(
    index='END_DATE', 
    columns='SEC_SHORT_NAME', 
    values='SUM_ALPHA_RET'
).plot()

In [ ]:
result.pivot_table(
    index='END_DATE', 
    columns='SEC_SHORT_NAME', 
    values='LOG_ALPHA_RET'
).mean(axis=1).expanding().sum().plot()

In [ ]:
result.pivot_table(
    index='END_DATE', 
    columns='SEC_SHORT_NAME', 
    values='LOG_ALPHA_RET'
).dropna().corr()

In [ ]:
df = dfs.get_fund_alpha_to_index(
    ticker_symbol='015868', 
    index_code='000852',
    start_date= '20220101'
)
df.index = pd.to_datetime(df["END_DATE"])

df["SUM_ALPHA_RET"].plot()

In [ ]:
query_sql = """
WITH a AS (
	SELECT
		END_DATE,
		avg( LOG_RET ) AS LOG_RET 
	FROM
		fund_adj_nav 
	WHERE
		1 = 1 
		AND TICKER_SYMBOL IN ( '005725', '006797', '006902', '005754', '006965', '007245', '006629', '531028', '006597', '002301', '006319' ) 
		AND END_DATE >= '20210101' 
	GROUP BY
		END_DATE 
	) SELECT
	END_DATE,
	exp( sum( LOG_RET / 100 ) over ( ORDER BY END_DATE ) ) AS NAV 
FROM
	a
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)
df = df.set_index("END_DATE")


In [ ]:
Performance(df["NAV"]).stats().T

In [ ]:
from portfolio.portfolio_backtest_new import PortfolioBacktest

pb = PortfolioBacktest([13, 21, 23])

In [ ]:
df = pb.get_backtest_nav().query("PORTFOLIO_NAME == '10'")

In [ ]:
cum_ret = df.pivot_table(index="TRADE_DT", columns="TEST_NAME", values="LOG_RETURN_RATE").dropna().cumsum()

In [ ]:
nav = np.exp(cum_ret/100)

In [ ]:
perf_list = []
for i in nav.columns:
    temp = Performance(nav[i]).stats().T
    temp["name"] = i
    perf_list.append(temp)
    

In [ ]:
perf_df = pd.concat(perf_list)

In [ ]:
perf_df

In [ ]:
nav.to_excel("D:/湖南分公司策略.xlsx")

In [ ]:
portfolio_holdings = pds.query_portfolo_holding_new()
fund_alpha = dfs.cal_fund_alpha_total_score(
    end_date="20231129", 
    period="1Y", 
    level_num=3,
    indicator_name_weights={"IR": 0.4, "ALPHA": 0.3, "MAXDD": 0.3, },
    indicator_period_weights={"3M": 0.2, "6M": 0.2, "1Y": 0.6, }
)

In [ ]:
portfolio_holdings = pds.query_portfolo_holding_new()
portfolio_holdings = portfolio_holdings[
    portfolio_holdings["PORTFOLIO_NAME"]
        .isin(["月月享", "均衡小确幸", "进取全明星"])
]

In [ ]:
portfolio_holdings = portfolio_holdings.merge(fund_alpha, how="left", on=["TICKER_SYMBOL"])

In [ ]:
portfolio_holdings.to_excel("d:/当前持仓.xlsx")

In [ ]:
fund_alpha = (
        fund_alpha.merge(dfs.query_fund_info(), how="left")
        .merge(dfs.query_fund_asset_own(), how="left")
        .merge(dfs.query_fund_type_own(), how="left")
        .merge(dfs.query_fund_fee(), how="left")
        .merge(dfs.query_basic_products(), how="left")
    )
fund_alpha.to_excel("D:/fund_alpha.xlsx")

In [ ]:
basic_products = dfs.query_basic_products().query("IF_IN_TRANCHE == 1")

In [ ]:
basic_products = basic_products.merge(fund_alpha, how="left")

In [ ]:
portfolio_products = pds.query_portfolio_products_ret("20231130")
fund_ret_rank = dfs.query_fund_ret_rank("20231130")

In [ ]:
(
    portfolio_products
    .merge(fund_ret_rank)
    .drop(columns=["PORTFOLIO_NAME"])
    .to_excel("d:/持仓表现.xlsx")
)

In [ ]:
query_sql = """
SELECT * FROM `chentiancheng`.`portfolio_performance` WHERE `END_DATE` = '20231130' AND `INDICATOR` = '累计收益率' ORDER BY `END_DATE` DESC LIMIT 0,1000
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)

In [ ]:
df.pivot_table(index="TICKER_SYMBOL", columns="CYCLE", values="BENCHMARK_VALUE_OUTTER")

In [ ]:
df = pd.read_csv(
    "E:/new_tdx/T0002/export/SZ#159531.csv", 
    skipfooter=1, 
    encoding='gbk', 
    header=None,
    names=["date", "time", "open", "high", "low", "close", "volume", "amount"]
)

df["date_time"] = pd.to_datetime(
    df["date"] + ' ' + df["time"].astype(str), format="%Y/%m/%d %H%M"
)
df.set_index("date_time", inplace=True)
df.drop(columns=["date", "time"],  inplace=True)


In [ ]:
df.resample("30min", closed="right", label="right").agg(
    {
        "open": "first",
        "high": "max",
        "low": "min",
        "close": "last",
        "volume": "sum",
        "amount": "sum",
    }
).dropna()

In [ ]:
df

In [ ]:
df["TIME"] = (
    df["TIME"].apply(lambda x: str(x).rjust(4, "0"))
    .apply(lambda x: x[:2]+":"+x[2:])
)

In [ ]:
df["DATE_TIME"] = df["DATE"] + ' ' + df["TIME"] 

In [ ]:
df.resample()

In [ ]:
file_path = "C:/Users/Wilcoxon/Desktop/!!!基金净值.xlsx"
df = pd.read_excel(file_path)
# df.set_index("DATE", inplace=True)
df

In [ ]:
year_list = [2019, 2020, 2021, 2022, 2023]
perf_list = []
for year in year_list:
    temp = df[df["DATE"].dt.year == year]
    temp.set_index("DATE", inplace=True)
    for col in temp.columns:
        temp_perf = Performance(temp[col]).stats().T
        temp_perf["YEAR"] = year
        temp_perf["组合"] = col
        perf_list.append(temp_perf)

In [ ]:
perf_df = pd.concat(perf_list)

In [ ]:
perf_df.pivot_table(index="YEAR", columns="组合", values="最大回撤")

In [ ]:
df = pds.query_portfolio_nav(portfolio_name="月月享")
df = df.set_index("TRADE_DT")

In [ ]:
temp_list = []
temp_list.append(Performance(df["PORTFOLIO_NAV"]).stats().T)
temp_list.append(Performance(df["BENCHMARK_NAV"]).stats().T)

In [ ]:
query_sql = """
SELECT
	* 
FROM
	`chentiancheng`.`portfolio_performance` 
WHERE
	`END_DATE` = '2023-11-30' 
	AND `TICKER_SYMBOL` = '月月享' 
	AND `CYCLE` IN ( 'YTD', '成立日', '近1年' ) 
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)

In [ ]:
df.pivot_table(index="INDICATOR", columns="CYCLE", values="PORTFOLIO_VALUE")